In [1]:
from google.cloud import storage
import os
import pandas as pd

In [2]:
pd.__version__

'0.24.0'

In [3]:
#os.environ['HTTPS_PROXY'] = "https://proxy-url:Port"
#os.environ['REQUESTS_CA_BUNDLE'] = "C:/Users/path/to/certs"
#os.environ["PROJECT_ID"] = "project-id-34914"

In [4]:
project=os.environ['PROJECT_ID']

In [5]:
client = storage.Client(project=project)

C:\Program Files\Anaconda3\envs\mnist\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
for b in client.list_buckets():
    print(b.name)

In [7]:
from google.cloud import bigquery
client = bigquery.Client(project=project)

In [8]:
query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100 * SUM(IF(answer_count > 0, 1, 0)) / COUNT(*), 1) AS Percent_Questions_with_Answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2016
ORDER BY
  Year
"""
#df = client.query(query).to_dataframe()
#df.head()
query_job = client.query(query)
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((2009, 343139, 99.7), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2010, 693332, 99.1), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2011, 1198587, 97.2), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2012, 1642687, 94.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2013, 2056613, 91.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2014, 2160361, 88.5), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2015, 2214389, 86.4), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})


In [9]:
print("Keys: ", tuple(row.keys()))
print("Values: ", row.values())

Keys:  ('Year', 'Number_of_Questions', 'Percent_Questions_with_Answers')
Values:  (2015, 2214389, 86.4)


In [10]:
df = client.query(query).to_dataframe()
df

,Year,Number_of_Questions,Percent_Questions_with_Answers
0,2009,343139,99.7
1,2010,693332,99.1
2,2011,1198587,97.2
3,2012,1642687,94.6
4,2013,2056613,91.6
5,2014,2160361,88.5
6,2015,2214389,86.4
